In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
loc = 'drive/My Drive/Signature_Verification'

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import random
from scipy import stats

from data_processing import *

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow.keras.layers import Dropout


from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate, Dropout
from tensorflow.keras.layers import BatchNormalization, MaxPool2D,Concatenate,Lambda,Flatten,Dense
from tensorflow.keras.initializers import glorot_uniform

from tensorflow.keras.models import Model

from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

%matplotlib inline

In [0]:
os.listdir()

['.config',
 '__pycache__',
 'BHSig260.zip',
 'data_processing.py',
 'drive',
 'sample_data']

# Dataset Generation

## Importing Data

In [0]:
imgsize = (150,150)
size= 150
path_dataset1 = os.path.join(loc,'Signature_Set1')
path_dataset2 = os.path.join(loc,'Signature_Set2')
path_dataset3 = 'Signature_Set3'
images_dictionary = {} # This dictionary stores all the images, to avoid too much memory consumption

In [0]:
fname = 'BHSig260.zip'

### Dataset from Set-1

In [0]:
real_img,forged_img = getImages(path_dataset1,imgsize,images_dictionary)
real_img = np.sort(real_img)
forged_img = np.sort(forged_img)
set1_X1,set1_X2,set1_Y = makePairs(real_img,forged_img,5)
print(len(set1_X1),len(set1_X2),len(set1_Y))

Getting Dataset-1 Data and Saving inside the Dictionary..
Getting Genuine Images..
25% Completed..
50% Completed..
100% Completed
Getting Forged Images..
25% Completed..
50% Completed..
100% Completed
600 600 600


### Dataset from Set-2

In [0]:
real_img,forged_img = getImages2(path_dataset2,imgsize,images_dictionary)
real_img = np.sort(real_img)
forged_img = np.sort(forged_img)
set2_X1,set2_X2,set2_Y = makePairs(real_img,forged_img,24)
print(len(set2_X1),len(set2_X2),len(set2_Y))

Getting Dataset2 Data..
25% Completed..
50% Completed..
Couldn't import  Thumbs.db in Location: drive/My Drive/Signature_Verification/Signature_Set2/full_org
100% Completed
25% Completed..
50% Completed..
Couldn't import  Thumbs.db in Location: drive/My Drive/Signature_Verification/Signature_Set2/full_forg
100% Completed
Data Import Complete!
30360 30360 30360


### Dataset from Set-3 (BHSig260)

In [0]:
real,forged = getHindi(fname,imgsize,images_dictionary)
X1H,X2H,yH  = makeHindiPairs(real,forged)

Getting Genuine Data..
25% Complete
50% Complete
75% Complete
100% Complete
Getting Forged Data..
25% Complete
50% Complete
75% Complete
100% Complete


Joining the Set2 and Set3 Datasets

In [0]:
X1 = np.concatenate([X1H,set2_X1])
X2 = np.concatenate([X2H,set2_X2])
Y  = np.concatenate([yH,set2_Y])
print(len(X1),len(X2),len(Y))

189720 189720 189720


#### Shuffling Data

In [0]:
import random
random.seed=35
mapIndexPosition = list(zip(X1,X2,Y))
random.shuffle(mapIndexPosition)
X1,X2,Y = zip(*mapIndexPosition)
Y = tf.cast(Y, tf.float32)

In [0]:
X1_train = X1[0:173720]
X2_train = X2[0:173720]
Y_train  = Y[0:173720]
X1_val   = X1[173720:]
X2_val   = X2[173720:]
Y_val    = Y[173720:]

## Generating Data

In [0]:
def create_batch(x1,x2,startpoint,batch_size):
    fir=np.zeros((batch_size,imgsize[0],imgsize[0],1))
    sec=np.zeros((batch_size,imgsize[0],imgsize[0],1))
    counter=0
    for i in range(startpoint,startpoint+batch_size):
        fir[counter]=images_dictionary[x1[i]]
        sec[counter]=images_dictionary[x2[i]]
        counter+=1
    return fir,sec

In [0]:
# Function takes in input of array of names, and returns the image list with expanded dimenstions
def returnImageList(name_lis):
    length = len(name_lis)
    imgs = np.zeros((length,imgsize[0],imgsize[0],1))
    for i in range(0,length):
        imgs[i] = images_dictionary[name_lis[i]]
    return imgs

In [0]:
def data_generator(batch_size,x1,x2,y):
    counter=0
    while True:
        if counter>=len(y): counter=0
        a,b = create_batch(x1,x2,counter,batch_size)
        y1 = y[counter:counter+batch_size]
        counter+=batch_size
        yield [a,b],y1

# Preparing the Model

### Embedding Model

In [0]:
emb_dim = 64

embedding_model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), input_shape=(size,size,1),name='Layer1'),
  tf.keras.layers.LeakyReLU(),
  tf.keras.layers.BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9),
  tf.keras.layers.MaxPooling2D(2, 2),
  #tf.keras.layers.Dropout(0.2),
    
  tf.keras.layers.Conv2D(128, (3,3),name='Layer2'),
  tf.keras.layers.LeakyReLU(),
  tf.keras.layers.BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Dropout(0.1),
    
  tf.keras.layers.Conv2D(256, (3,3),name='Layer3'),
  tf.keras.layers.LeakyReLU(),
  tf.keras.layers.BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9),
  tf.keras.layers.MaxPooling2D(4, 4),
  tf.keras.layers.Dropout(0.2),

  tf.keras.layers.Conv2D(512, (3,3),name='Layer4'),
  tf.keras.layers.LeakyReLU(),
  tf.keras.layers.BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9),
  tf.keras.layers.MaxPooling2D(4, 4),
  tf.keras.layers.Dropout(0.1),

  tf.keras.layers.Conv2D(512, (1,1),name='Layer5'),
  tf.keras.layers.LeakyReLU(),
  tf.keras.layers.Flatten(),
],name='embedding_model')
print(embedding_model.summary())

Model: "embedding_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Layer1 (Conv2D)              (None, 148, 148, 64)      640       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 148, 148, 64)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 148, 148, 64)      592       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 64)        0         
_________________________________________________________________
Layer2 (Conv2D)              (None, 72, 72, 128)       73856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 72, 72, 128)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 72, 72, 128)   

### Siamese-Type Network

In [0]:
input_a = tf.keras.layers.Input(shape=(size,size,1),name='input1')
input_b = tf.keras.layers.Input(shape=(size,size,1),name='input2')

em_one = embedding_model(input_a)
em_two = embedding_model(input_b)

out = tf.keras.layers.concatenate([em_one,em_two],axis=1)
out = tf.keras.layers.Dense(512,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(l=0.1))(out)
out = tf.keras.layers.Dense(64,activation='relu')(out)

out = tf.keras.layers.Dense(1,activation='sigmoid',name='Output')(out)

model = tf.keras.models.Model([input_a, input_b],out)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input1 (InputLayer)             [(None, 150, 150, 1) 0                                            
__________________________________________________________________________________________________
input2 (InputLayer)             [(None, 150, 150, 1) 0                                            
__________________________________________________________________________________________________
embedding_model (Sequential)    (None, 512)          1813520     input1[0][0]                     
                                                                 input2[0][0]                     
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 1024)         0           embedding_model[1][0]        

### Learning Rate Scheduling

In [0]:
initial_learning_rate = 0.0008
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=5, 
    decay_rate=0.96,
    staircase=True
) #lr = lr * (decay_rate ^ decay_steps)
optimizer= tf.keras.optimizers.RMSprop(learning_rate=lr_schedule)

In [0]:
batch_size = 8
epoch = 4
rms = tf.keras.optimizers.RMSprop(lr=1e-4, rho=0.9, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy'])

In [0]:
_ =model.fit(
    data_generator(batch_size,X1_train,X2_train,Y_train),
    epochs=epoch,
    steps_per_epoch=len(Y_train) // batch_size,
    validation_data=data_generator(batch_size,X1_val,X2_val,Y_val),
    validation_steps = len(Y_val) // batch_size,
    verbose=1
)

Epoch 1/4
21715/21715 [==============================] - 2165s 100ms/step - loss: 0.6043 - accuracy: 0.7661 - val_loss: 0.5550 - val_accuracy: 0.7197
Epoch 2/4
21715/21715 [==============================] - 2170s 100ms/step - loss: 0.5252 - accuracy: 0.7683 - val_loss: 0.5549 - val_accuracy: 0.7199
Epoch 3/4
21715/21715 [==============================] - 2188s 101ms/step - loss: 0.5254 - accuracy: 0.7677 - val_loss: 0.5549 - val_accuracy: 0.7199
Epoch 4/4
21715/21715 [==============================] - 2192s 101ms/step - loss: 0.5253 - accuracy: 0.7680 - val_loss: 0.5548 - val_accuracy: 0.7200


# Evaluation

### On Training Set
*(Accuracy suggests that the model has avoided overfitting along with learning)*

In [0]:
train_set_accuracy = model.evaluate(data_generator(batch_size,X1_train,X2_train,Y_train),batch_size=batch_size,steps=len(Y_val) // batch_size)

2000/2000 [==============================] - 75s 37ms/step - loss: 0.5554 - accuracy: 0.7153


In [0]:
print("Training Set Accuracy is : ", train_set_accuracy[1]*100)

Training Set Accuracy is :  71.53124809265137


### On Validation Set
*(Accuracy suggests how well the model has actually learnt)*

In [0]:
validation_set_accuracy=model.evaluate(data_generator(batch_size,X1_val,X2_val,Y_val),batch_size=batch_size,steps=len(Y_val) // batch_size)

2000/2000 [==============================] - 75s 37ms/step - loss: 0.5548 - accuracy: 0.7200


In [0]:
print("Validation Set Accuracy = ",validation_set_accuracy[1]*100)

Validation Set Accuracy =  72.00000286102295


### On Dataset which is totally new for the model
*(Expected Accuracy is very low, just to get an idea on how well the model has learnt)* <br> <br>
**NOTE: This dataset is not considered as test set, as this dataset(Containing full english signatures) is not similar to the training set(Mostly having Hindi Language Signatures).** <br><br>
This Accuracy is calculated based on threshold. Since the result are the probability (by sigmoid activation function), threshold using the **mean** of all the values gave comparatively better accuracy.

In [0]:
test_preds = model.predict([returnImageList(set1_X1),returnImageList(set1_X2)])

In [0]:
def calculateAccuracy(Y,predictions,threshold): # Calculates Accuracy on a given threshold value
    test_preds_final = []
    for i in predictions:
        if i[0]<0.35: test_preds_final.append(0)
        else: test_preds_final.append(1)

    counter=0
    for i in range(0,len(set1_Y)):
        if set1_Y[i] == test_preds_final[i]:
            counter+=1
    return counter*100/len(test_preds_final)

In [0]:
mean = np.mean(test_preds)
print(mean)

0.49748087


In [0]:
print("Accuracy gained by the totally new dataset is : ",calculateAccuracy(set1_Y,test_preds,np.mean(test_preds)))

Accuracy gained by the totally new dataset is :  57.5
